<a href="https://colab.research.google.com/github/Aarthi0705/Aarthi0705/blob/main/Medical_rag_works_with_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers sentence-transformers faiss-cpu datasets rank_bm25 accelerate bitsandbytes scispacy
!pip install -U pip setuptools wheel
!pip install -U 'spacy[cuda11x,transformers,lookups]'
!python -m spacy download en_core_web_sm
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.4/en_core_sci_sm-0.5.4.tar.gz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.2/920.2 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 26.0 MB/s eta 0:00:00
  Attempting uninstall: blis
    Found existing installation: blis 1.2.0
    Uninstalling blis-1.2.0:
      Successfully uninstalled blis-1.2.0
  Attempting uninstall: thinc
    Found existing installation: thinc 8.3.4
    Uninstalling thinc-8.3.4:
      Successfully uninstalled thinc-8.3.4
  Attempting uninstall: spacy
    Found existing installation: spacy 3.8.4
    Uninstalling spacy-3.8.4:
      Successfully uninstalled spacy-3.8.4
  Using cached spacy-3.8.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (27 kB)
  Using cached thinc-8.3.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (15 kB)
  Using cached spacy_transformers-1.3.8-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.0 kB)


**Load Medical books**

In [ ]:
!pip install --upgrade datasets
from datasets import load_dataset

# Load medical text dataset (PubMed abstracts) - try a different, similar dataset as 'allenai/pubmedqa' might not be public or available
dataset = load_dataset("scientific_papers", "pubmed", split="train")

# Extract only the text field
texts = [item['article'] for item in dataset] #use 'article' key for text

print("Sample Text:", texts[0])
print("Total Documents:", len(texts))

README.md:   0%|          | 0.00/8.27k [00:00<?, ?B/s]

scientific_papers.py:   0%|          | 0.00/5.35k [00:00<?, ?B/s]

The repository for scientific_papers contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/scientific_papers.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/119924 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/6633 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6658 [00:00<?, ? examples/s]

Sample Text: a recent systematic analysis showed that in 2011 , 314 ( 296 - 331 ) million children younger than 5 years were mildly , moderately or severely stunted and 258 ( 240 - 274 ) million were mildly , moderately or severely underweight in the developing countries .
in iran a study among 752 high school girls in sistan and baluchestan showed prevalence of 16.2% , 8.6% and 1.5% , for underweight , overweight and obesity , respectively .
the prevalence of malnutrition among elementary school aged children in tehran varied from 6% to 16% .
anthropometric study of elementary school students in shiraz revealed that 16% of them suffer from malnutrition and low body weight .
snack should have 300 - 400 kcal energy and could provide 5 - 10 g of protein / day . nowadays , school nutrition programs are running as the national programs , world - wide . national school lunch program in the united states
there are also some reports regarding school feeding programs in developing countries . 

# Implement Adaptive Chunking
Now, we will split text intelligently using:

Token Density (more words → smaller chunks) <br>
Semantic Entropy (important words → finer splits) <br>
Medical Entity Frequency (UMLS terms → finer splits) <br>
Graph Centrality (higher importance → finer splits)


✅Handles Empty Texts <→ Skips them before processing.<br>
✅ Avoids TF-IDF Errors → Uses stop_words="english" and checks if matrix is empty.<br>
✅ Improves Medical NLP → Uses en_core_sci_sm for better entity recognition.


In [ ]:
import nltk
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# Download NLTK tokenizer
nltk.download('punkt')
nltk.download('punkt_tab')

# Load SpaCy medical model (Better for medical NLP)
try:
    nlp = spacy.load("en_core_sci_sm")  # Use "en_core_sci_md" for better results
except:
    nlp = spacy.load("en_core_web_sm")  # Fallback if sci model is missing

# Function to calculate chunk importance
def chunk_importance(text):
    if not text.strip():  # Skip empty or whitespace-only text
        return 0

    # Token Density
    tokens = nltk.word_tokenize(text)
    density = len(tokens) / (len(text) + 1)  # Avoid division by zero

    # Semantic Entropy (TF-IDF)
    try:
        vectorizer = TfidfVectorizer(stop_words="english")  # Remove stopwords
        tfidf_matrix = vectorizer.fit_transform([text])
        entropy = np.mean(tfidf_matrix.toarray()) if tfidf_matrix.shape[1] > 0 else 0
    except ValueError:
        entropy = 0  # If TF-IDF fails, set entropy to 0

    # Medical Entity Frequency
    doc = nlp(text)
    medical_entities = len([ent for ent in doc.ents if ent.label_ in ["DISEASE", "TREATMENT", "MEDICATION"]])

    # Combined Score
    return 0.4 * density + 0.3 * entropy + 0.3 * medical_entities

# Adaptive chunking function
def adaptive_chunk(text, max_length=256):
    sentences = nltk.sent_tokenize(text)
    chunks = []
    current_chunk = []

    for sent in sentences:
        current_chunk.append(sent)
        chunk_text = " ".join(current_chunk)
        if len(chunk_text.split()) > max_length or chunk_importance(chunk_text) > 0.5:
            chunks.append(chunk_text)
            current_chunk = []

    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks

# Remove empty texts
filtered_texts = [text for text in texts if text.strip()]

# Apply adaptive chunking
chunked_texts = [adaptive_chunk(text) for text in filtered_texts[:100]]  # Process first 100 docs

print("Sample Chunk:", chunked_texts[0])
print("Total Chunks Created:", sum(len(chunks) for chunks in chunked_texts))


/usr/local/lib/python3.11/dist-packages/cupy/_environment.py:447: UserWarning: 
--------------------------------------------------------------------------------

  CuPy may not function correctly because multiple CuPy packages are installed
  in your environment:

    cupy-cuda11x, cupy-cuda12x

  Follow these steps to resolve this issue:

    1. For all packages listed above, run the following command to remove all
       existing CuPy installations:

         $ pip uninstall <package_name>

      If you previously installed CuPy via conda, also run the following:

         $ conda uninstall cupy

    2. Install the appropriate CuPy package.
       Refer to the Installation Guide for detailed instructions.

         https://docs.cupy.dev/en/stable/install.html

--------------------------------------------------------------------------------

  warnings.warn(f'''
/usr/local/lib/python3.11/dist-packages/cupy/_environment.py:447: UserWarning: 
--------------------------------------------

0it [00:00, ?it/s]

/usr/local/lib/python3.11/dist-packages/spacy/language.py:2195: FutureWarning: Possible set union at position 6328
  deserializers["tokenizer"] = lambda p: self.tokenizer.from_disk(  # type: ignore[union-attr]


Sample Chunk: ['a recent systematic analysis showed that in 2011 , 314 ( 296 - 331 ) million children younger than 5 years were mildly , moderately or severely stunted and 258 ( 240 - 274 ) million were mildly , moderately or severely underweight in the developing countries . in iran a study among 752 high school girls in sistan and baluchestan showed prevalence of 16.2% , 8.6% and 1.5% , for underweight , overweight and obesity , respectively . the prevalence of malnutrition among elementary school aged children in tehran varied from 6% to 16% . anthropometric study of elementary school students in shiraz revealed that 16% of them suffer from malnutrition and low body weight . snack should have 300 - 400 kcal energy and could provide 5 - 10 g of protein / day . nowadays , school nutrition programs are running as the national programs , world - wide . national school lunch program in the united states\nthere are also some reports regarding school feeding programs in developing countrie

In [ ]:
!pip install sentence-transformers faiss-cpu scikit-learn

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np

# Load a domain-specific embedding model
biomed_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Sample check
test_embedding = biomed_model.encode("This is a test sentence.")
print("Embedding Shape:", test_embedding.shape)  # Should be (384,) or similar


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embedding Shape: (384,)


Generate Dense and Sparse Embeddings

In [ ]:
from sentence_transformers import SentenceTransformer

# Load dense embedding model (use any compatible model)
dense_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")  # 384-dim embeddings


In [ ]:
import faiss
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.decomposition import TruncatedSVD


# Initialize TF-IDF Vectorizer
vectorizer = TfidfVectorizer(max_features=5000)  # Limit features to 5000 for efficiency
# Fit on the flattened text chunks

from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import normalize
import numpy as np
import numpy as np
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import normalize

def generate_hybrid_embeddings(texts, target_dim=484):
    global vectorizer  # Ensure vectorizer is trained before calling this function

    dense_embeddings = dense_model.encode(texts, normalize_embeddings=True)  # (N, 384)

    # 🔹 Ensure `sparse_embeddings` is always defined
    if not hasattr(vectorizer, "vocabulary_"):  # Check if vectorizer is fitted
        raise ValueError("❌ Vectorizer not trained! Run `vectorizer.fit(corpus)` before using it.")

    sparse_embeddings = vectorizer.transform(texts).toarray()  # (N, 5000)

    # 🚨 Handle empty sparse embeddings
    if np.all(sparse_embeddings == 0):
        print("⚠ Warning: All sparse embeddings are zero! Adding small noise to avoid SVD failure.")
        sparse_embeddings += np.random.normal(0, 1e-6, sparse_embeddings.shape)

    sparse_target_dim = target_dim - dense_embeddings.shape[1]  # 484 - 384 = 100

    # 🔹 Ensure TruncatedSVD doesn't produce NaN values
    try:
        svd = TruncatedSVD(n_components=sparse_target_dim)
        reduced_sparse = svd.fit_transform(sparse_embeddings)  # (N, 100)

        if np.isnan(reduced_sparse).any():
            print("❌ NaN values in SVD output! Replacing with zeros.")
            reduced_sparse = np.nan_to_num(reduced_sparse)  # Replace NaNs with 0s

    except ValueError as e:
        print("❌ TruncatedSVD failed:", e)
        reduced_sparse = np.zeros((sparse_embeddings.shape[0], sparse_target_dim))  # Fallback to zeros

    # Normalize and combine embeddings
    dense_norm = normalize(dense_embeddings, axis=1)
    sparse_norm = normalize(reduced_sparse, axis=1)

    hybrid_embeddings = np.hstack([dense_norm, sparse_norm])  # (N, 484)

    print(f"✅ Hybrid Embeddings Shape: {hybrid_embeddings.shape}")  # Debugging
    return hybrid_embeddings

# Apply to our chunked data
flat_chunks = [" ".join(chunks) for chunks in chunked_texts]
vectorizer.fit(flat_chunks)  # Flatten chunked texts
hybrid_embeddings = generate_hybrid_embeddings(flat_chunks)

print("Hybrid Embeddings Shape:", hybrid_embeddings.shape)

✅ Hybrid Embeddings Shape: (100, 484)
Hybrid Embeddings Shape: (100, 484)


In [ ]:
!pip install faiss-cpu

In [ ]:
import faiss
import numpy as np
# Initialize FAISS index
index = faiss.IndexFlatL2(484)  # Ensure this matches the target_dim (484)
# Convert embeddings to float32 before adding to FAISS
index.add(hybrid_embeddings.astype(np.float32))

print(f"✅ FAISS index created with {index.ntotal} entries.")


✅ FAISS index created with 100 entries.


Search function

In [ ]:
def search(query, top_k=3):
    query_embedding = generate_hybrid_embeddings([query])  # Generate query embedding
    query_dim = query_embedding.shape[1]  # Get the actual dimension of query embedding
    faiss_dim = index.d  # Get the expected FAISS index dimension

    print(f"Query embedding shape: {query_embedding.shape}")
    print(f"FAISS index expected shape: ({faiss_dim},)")

    # 🚀 Handle Shape Mismatch
    if query_dim != faiss_dim:
        print(f"❌ Shape Mismatch: Query ({query_dim}) vs FAISS ({faiss_dim})")

        if query_dim < faiss_dim:
            # **Case 1: Query is smaller → Zero Padding**
            padding = np.zeros((1, faiss_dim - query_dim))  # Add zero padding
            query_embedding = np.hstack([query_embedding, padding])
        else:
            # **Case 2: Query is larger → Trim the excess**
            query_embedding = query_embedding[:, :faiss_dim]

        print(f"✅ Fixed Query Shape: {query_embedding.shape}")

    # Perform FAISS search
    distances, indices = index.search(query_embedding.astype(np.float32), top_k)

    # Retrieve matched text chunks
    results = [flat_chunks[idx] for idx in indices[0] if idx != -1]  # Ignore invalid indices

    return results


In [ ]:
query = "What are the symptoms of leukemia?"
results = search(query, top_k=3)

print("\n🔎 Search Results:")
for i, res in enumerate(results):
    print(f"{i+1}. {res[:200]}...")


✅ Hybrid Embeddings Shape: (1, 385)
Query embedding shape: (1, 385)
FAISS index expected shape: (484,)
❌ Shape Mismatch: Query (385) vs FAISS (484)
✅ Fixed Query Shape: (1, 484)

🔎 Search Results:
1. health is not only related to the absence of the disease , therefore we need to conceptualize and operationalize what health is . increasingly , we have come to understand that information about funct...
2. cystic echinococcosis ( ce ) is a severe zoonosis caused by the cyclophyllidean cestode echinococcus granulosus . the disease has a worldwide distribution , with endemic regions in many countries of t...
3. polycystic ovary syndrome ( pcos ) which was first reported in 1935 is known as one of the most common endocrine hormones disorders in the women of the reproductive age afflicting as many as % 10 of t...


/usr/local/lib/python3.11/dist-packages/sklearn/decomposition/_truncated_svd.py:273: RuntimeWarning: invalid value encountered in divide
  self.explained_variance_ratio_ = exp_var / full_var


In [ ]:
!pip install transformers accelerate sentencepiece

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Choose an open-source LLM (change to 'mistralai/Mistral-7B-v0.1' if needed)
model_name = "meta-llama/Llama-2-7b-chat-hf"

# Load tokenizer & model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,  # Use float16 for efficiency
    device_map="auto"  # Auto GPU/CPU allocation
)

print("✅ LLM Loaded Successfully!")


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

✅ LLM Loaded Successfully!


Model Generator

In [ ]:
import torch
print("CUDA Available:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU Found")


CUDA Available: False
GPU: No GPU Found


In [19]:
def generate_answer(query, top_k=5, max_new_tokens=150):
    # Step 1: Retrieve relevant chunks
    retrieved_chunks = search(query, top_k)
    context = "\n".join(retrieved_chunks)

    # Step 2: Limit the context length
    max_context_tokens = 400  # Adjust based on model capacity
    tokenized_context = tokenizer(context, truncation=True, max_length=max_context_tokens)
    trimmed_context = tokenizer.decode(tokenized_context["input_ids"], skip_special_tokens=True)

    # Step 3: Format the prompt
    prompt = f"""
    You are a helpful assistant. Answer the question using the provided context.

    Context:
    {trimmed_context}

    Question: {query}
    Answer:
    """

    # Step 4: Tokenize and Generate Response

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    output = model.generate(**inputs, max_new_tokens=max_new_tokens)
    response = tokenizer.decode(output[0], skip_special_tokens=True)

    return response

# Test the updated function
query = "What are the symptoms of diabetes?"
response = generate_answer(query)

print("\n🤖 LLM Response:\n", response)


/usr/local/lib/python3.11/dist-packages/sklearn/decomposition/_truncated_svd.py:273: RuntimeWarning: invalid value encountered in divide
  self.explained_variance_ratio_ = exp_var / full_var


✅ Hybrid Embeddings Shape: (1, 385)
Query embedding shape: (1, 385)
FAISS index expected shape: (484,)
❌ Shape Mismatch: Query (385) vs FAISS (484)
✅ Fixed Query Shape: (1, 484)


RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

Optimize Performance (Speed & Memory Efficiency)<br>
Now that the RAG pipeline is working, let's:<br>
✅ Reduce model memory usage (quantization)<br>
✅ Speed up response time (batch processing)

In [ ]:
!pip install bitsandbytes transformers accelerate

In [ ]:
from transformers import BitsAndBytesConfig, AutoModelForCausalLM

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,  # ✅ Use float16
    bnb_4bit_use_double_quant=True
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    torch_dtype=torch.float16,  # ✅ Ensure model tensors use float16
    device_map="auto"
)

print("✅ Model Loaded with 4-bit Quantization (Float16)")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ Model Loaded with 4-bit Quantization (Float16)


In [ ]:
import torch

def generate_answer(query, top_k=5, max_new_tokens=150):
    # Step 1: Retrieve relevant chunks
    retrieved_chunks = search(query, top_k)
    context = "\n".join(retrieved_chunks)

    # Step 2: Limit the context length
    max_context_tokens = 400  # Adjust based on model capacity
    tokenized_context = tokenizer(context, truncation=True, max_length=max_context_tokens)
    trimmed_context = tokenizer.decode(tokenized_context["input_ids"], skip_special_tokens=True)

    # Step 3: Format the prompt
    prompt = f"""
    You are a helpful assistant. Answer the question using the provided context.

    Context:
    {trimmed_context}

    Question: {query}
    Answer:
    """

    # Step 4: Tokenize input and move to GPU
    inputs = tokenizer(prompt, return_tensors="pt")

    # Convert tensors to float16 manually (only for model input tensors)
    for key in inputs:
        inputs[key] = inputs[key].to("cuda")

    # Generate response
    with torch.no_grad():  # Ensure inference mode for efficiency
        output = model.generate(**inputs, max_new_tokens=max_new_tokens)

    response = tokenizer.decode(output[0], skip_special_tokens=True)

    return response

# Test the function
query = "What are the symptoms of diabetes?"
response = generate_answer(query)

print("\n🤖 LLM Response:\n", response)


/usr/local/lib/python3.11/dist-packages/sklearn/decomposition/_truncated_svd.py:273: RuntimeWarning: invalid value encountered in divide
  self.explained_variance_ratio_ = exp_var / full_var


✅ Hybrid Embeddings Shape: (1, 385)
Query embedding shape: (1, 385)
FAISS index expected shape: (484,)
❌ Shape Mismatch: Query (385) vs FAISS (484)
✅ Fixed Query Shape: (1, 484)

🤖 LLM Response:
 
    You are a helpful assistant. Answer the question using the provided context.

    Context:
    to review the current knowledge about nonpharmacologic approaches in the prevention and early treatment of type 2 diabetes . this study reviewed the research reports dealing with nonpharmacologic interventions aimed at preventing type 2 diabetes with early lifestyle interventions . the results from the randomized controlled trials all show that people with impaired glucose tolerance who received enhanced lifestyle advice had significantly lower ( on average 50% reduced ) incidence of type 2 diabetes compared with those allocated to receive  usual care . individuals who were able to correct their lifestyle habits as recommended for usual healthy life patterns were mostly protected against type 2 

In [ ]:
def batch_generate_answers(queries, top_k=5, max_new_tokens=150):
    results = []

    for query in queries:
        response = generate_answer(query, top_k, max_new_tokens)
        results.append({"query": query, "response": response})

    return results



queries = ["What are the symptoms of diabetes?", "How to treat hypertension?", "What is insulin resistance?"]
responses = batch_generate_answers(queries)

for r in responses:
    print(f"\n❓ Query: {r['query']}\n🤖 Response: {r['response']}")



/usr/local/lib/python3.11/dist-packages/sklearn/decomposition/_truncated_svd.py:273: RuntimeWarning: invalid value encountered in divide
  self.explained_variance_ratio_ = exp_var / full_var


✅ Hybrid Embeddings Shape: (1, 385)
Query embedding shape: (1, 385)
FAISS index expected shape: (484,)
❌ Shape Mismatch: Query (385) vs FAISS (484)
✅ Fixed Query Shape: (1, 484)


/usr/local/lib/python3.11/dist-packages/sklearn/decomposition/_truncated_svd.py:273: RuntimeWarning: invalid value encountered in divide
  self.explained_variance_ratio_ = exp_var / full_var


✅ Hybrid Embeddings Shape: (1, 385)
Query embedding shape: (1, 385)
FAISS index expected shape: (484,)
❌ Shape Mismatch: Query (385) vs FAISS (484)
✅ Fixed Query Shape: (1, 484)


/usr/local/lib/python3.11/dist-packages/sklearn/decomposition/_truncated_svd.py:273: RuntimeWarning: invalid value encountered in divide
  self.explained_variance_ratio_ = exp_var / full_var


✅ Hybrid Embeddings Shape: (1, 385)
Query embedding shape: (1, 385)
FAISS index expected shape: (484,)
❌ Shape Mismatch: Query (385) vs FAISS (484)
✅ Fixed Query Shape: (1, 484)

❓ Query: What are the symptoms of diabetes?
🤖 Response: 
    You are a helpful assistant. Answer the question using the provided context.

    Context:
    to review the current knowledge about nonpharmacologic approaches in the prevention and early treatment of type 2 diabetes . this study reviewed the research reports dealing with nonpharmacologic interventions aimed at preventing type 2 diabetes with early lifestyle interventions . the results from the randomized controlled trials all show that people with impaired glucose tolerance who received enhanced lifestyle advice had significantly lower ( on average 50% reduced ) incidence of type 2 diabetes compared with those allocated to receive  usual care . individuals who were able to correct their lifestyle habits as recommended for usual healthy life pattern

In [ ]:
from datasets import load_metric
import numpy as np

# Load standard NLP metrics
rouge = load_metric("rouge")
bleu = load_metric("sacrebleu")
meteor = load_metric("meteor")

# Example ground truth and generated responses
references = ["Diabetes symptoms include frequent urination, increased thirst, and fatigue."]
predictions = [generate_answer("What are the symptoms of diabetes?")]

# Compute ROUGE scores
rouge_scores = rouge.compute(predictions=predictions, references=references)

# Compute BLEU score
bleu_score = bleu.compute(predictions=predictions, references=[[ref] for ref in references])

# Compute METEOR score
meteor_score = meteor.compute(predictions=predictions, references=references)

print("ROUGE Scores:", rouge_scores)
print("BLEU Score:", bleu_score)
print("METEOR Score:", meteor_score)


ModuleNotFoundError: No module named 'datasets'